# Spotify Genre Classification 

## Getting the Data - Scraping and Spotify API

### explaining the functions of API:
the target : to create a dataFrame that contains the information from the billabord charts and from spotify. the information from spotify is the id of each song that is in the billaboard chart.

We get the id of the song,which is the connection between the billabord info and the spotify info (we take each song and artist name from the billabord chart and search for the specific song id in order to get the features that we want to check on each song, in order to classify which song is a hit or not , i.e. which song is popular and which isn't (by the popularity column, which we turned from numeric to catagorial (yes/no).

In addition to getting the id - with get_id_of_songs() function, that uses other functions : get_key(val) and find_id(song,result), this function also returns the uri , the song features that we will use in the dataFrame and also info about the artist ( with find_artist_info(result) ) , which includes the 'popularity' feature , which is out target variable.  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize
import requests 
import scipy.stats as stats
import datetime
import base64
clientid = '4d7f3ed374354cbf848b72bc270abedd'
secretid = '7c9935d59cbc417e8ac74ad8d7e06a6e'
table_MN = pd.read_html('https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2019')
print(f'Total tables: {len(table_MN)}')
df1 = table_MN[0]
table_MN = pd.read_html('https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2018')
df2 = table_MN[0]
table_MN = pd.read_html('https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2017')
df3 = table_MN[0]
table_MN = pd.read_html('https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2016')
df4 = table_MN[0]
table_MN = pd.read_html('https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2015')
df5 = table_MN[0]
table_MN = pd.read_html('https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2014')
df6 = table_MN[0]
table_MN = pd.read_html('https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2013')
df7 = table_MN[1]
table_MN = pd.read_html('https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2012')
df8 = table_MN[1]
table_MN = pd.read_html('https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2011')
df9 = table_MN[0]
frames = [df1, df2, df3, df4, df5, df6, df7, df8, df9]
frames = [df1]

## todo: get more data, and really scrape this.
all_songs_list = pd.concat(frames)

Total tables: 2


In [ ]:
# print(all_songs_list)

In [ ]:
# sp.search("lsdjlk asfklfafas", limit=1,type='track')

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
cid = "3b41c863d85746899b1bd17a39149f8e"
secret = "631b736a77d545e4a822bea470435cea"

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
import re

song_results = []
for index, row in all_songs_list.iterrows():
    song_rank = row['No.']
    artist_original_name = row['Artist(s)']
    artist_names = re.split('featuring | and | ,',artist_original_name)
    artist_names.append(artist_original_name)
    for name in artist_names:
        query = "track:"+row['Title']+" artist:"+name
        query = query.replace('"','')
        search_result = sp.search(query, limit=1,type='track')
        if len(search_result['tracks']['items']) != 0:
            song_entity ={'rank':song_rank,'result':search_result}
            song_results.append(song_entity)
            break
#         song_queries.append(query)

#### get the artist/band name , in order to search for the song title in the find_id() function

In [ ]:
# so we will get an additinal info about the artist and the song - genre , popularity and etc.
def find_artist_info(result):
    for data in result['tracks']['items']:
        for key,value in data.items():
                if (key=='artists'): 
                      return (value[0]['id'])  


In [ ]:
#find_artist_info(result)

In [ ]:
def get_song_features(song_entity):
    search_result = song_entity['result']
    track_uri = search_result['tracks']['items'][0]['uri']
    track_features = sp.audio_features(track_uri)[0]
    artist_id = search_result['tracks']['items'][0]['artists'][0]['id']
    artist_info = sp.artist(artist_id)
    artist_popularity = artist_info['popularity']
    artist_genre = artist_info['genres'][0]
    
    features = track_features
    features['genre'] = artist_genre
    features['popularity'] = artist_popularity
    features['rank'] = song_entity['rank']

    return features

In [ ]:
# constant1 = "track:Overshadows artist:Shredhead"
# get_song_features(constant1)

def get_songs_df(song_entities):
    song_list = []
    for ent in song_entities:
        try:
            song_features = get_song_features(ent)
            song_list.append(song_features)
        except:
            print(ent['result'])
            print("song was not found!")
    
    return pd.DataFrame(song_list)

# Introduction to the problem

### What is Classification?

A classification algorithm takes a dataset of labelled examples as inputs to produce a model that can take unlabeled new data and automatically assign labels to the unlabeled example.

If the classification problem has a set of two labels (for instance “spam” or “not spam”) then it is a binary classification problem. Multi-class classification is a problem where the number of labels within the set is three or greater. 

The problem that we are looking at is a multi-class as there are many genres within the set.

### our independent features/variables: 

#### A. from spotify API track info:

#### B. from spotify API artist info:

#### C. from billaboard charts' scraping: 
     rank- song position
     

## What is the target column?
our target column (dependent feature): genre
    
genre is the music type which the song is accosiated to
in the data aquisition process, we only took the rifst genre the song is accosiated to.

our aprior ideas:
   
   * the features 'danceability', 'energy', loudness ,instrumentalness ,speechiness, liveness', pop and tempo have high        correlation to the genre feature, so they are good features to use in order to classify the songs' genre.
    this is due to the fact that those features are linked very strongl to specific genres.
    
for example, a pop song has a big likelihood to have high danceability, energy, loudness and pop (popularity), whereas a rock song has a big likelihood to have high loudness and instrumentalness, in comparison to a country song that is likely to have low loudness and high acousticness	.

* the rank feature may be helpful to classify the songs' genre since we know that most of the song that has high ranking are classified as pop songs or rap/hip hop songs.


#  getting to know the data

In [ ]:
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline

#for preprocessing part
from scipy import stats

In [ ]:
#DataFrame of all our songs
songs_df = get_songs_df(song_results)

In [ ]:
songs_df

#  preprocessing of the data

In [ ]:
songs_df.head()

## dealing with null values

In [ ]:
songs_df.isnull().any()

In [ ]:
# # import missingno as msno
# # msno.bar(songs_df, color="dodgerblue", sort="ascending", figsize=(10,5), fontsize=12)
# songs_df["genre"].isnull().value_counts()
# dropping NULL values
songs_df = songs_df.dropna(axis=0)

In [ ]:
songs_df["genre"].isnull().value_counts()

we see that there are a lot of dirrerent genres, so we will reduce them: 

In [ ]:
# coverting sub-genres to main genre
def replace_genre(genre,song,index,songs_df):
    if genre in song['genre']:
        song['genre'] = genre
        songs_df.iloc[index] = song

In [ ]:
# calling replace_genre method
for index, song in songs_df.iterrows():
    replace_genre("hip hop",song,index,songs_df)
    replace_genre("rap",song,index,songs_df)
    replace_genre("pop",song,index,songs_df)
    replace_genre("rock",song,index,songs_df)
    replace_genre("country",song,index,songs_df)


In [ ]:
# to make sure we didn't miss any genre
songs_df = songs_df.replace({'genre': {"album rock": "rock", "glam rock": "rock", "dance rock":"rock", "art rock":"rock",
                                  "soft rock":"rock", "country rock": "rock", "classic rock":"rock", "blues rock":"rock", "celtic rock":"rock",
                                  "australian rock":"rock", "german alternative rock":"rock", "alternative rock":"rock", "dance pop":"pop",
                                "brill building pop": "pop", "europop": "pop", "barbadian pop": "pop", "classic uk pop":"pop", 
                                "new wave pop":"pop", "canadian pop":"pop", "art pop":"pop", "belgian pop": "pop", "britpop": "pop", 
                                "italian pop":"pop", "classic danish pop": "pop", "bow pop": "pop", "baroque pop": "pop", "bubblegum pop": "pop",
                                "afropop":"pop", "hip pop":"pop", "atl hip hop": "hip hop", "east coast hip hop": "hip hop", "detroit hip hop":"hip hop", 
                                "bronx hip hop": "hip hop", "bubblegum dance": "dance", "eurodance":"dance", "belgian dance":"dance", "german dance": "dance",
                                "classic soul": "soul", "british soul": "soul", "chicago soul": "soul", "british folk": "folk", "american folk revival":"folk",
                                "drone folk":"folk","canadian folk":"folk", "deep adult standards":"adult standards", "glam metal": "metal", "alternative metal": "metal",
                                "acoustic blues":"blues", "british blues":"blues", "louisiana blues":"blues", "g funk":"funk", "brit funk":"funk",
                                "afrobeat":"dance", "british invasion":"rock", "doo-wop":"blues", "boy band":"pop", "merseybeat":"rock-and-roll", "blue":"blues",
                                                "bebop":"jazz", "avant-garde jazz":"jazz", "boogaloo": "latin", "big room": "trance", "bubble trance":"trance", "glam punk":"rock",
                                                "australian talent show":"pop", "mellow gold":"rock", "hi-nrg": "dance", "neo mellow": "pop", "yodeling":"folk", "classic girl group":"pop",
                                                "british dance band":"jazz", "deep house":"dance", "uk garage": "dance", "chicago rap":"hip hop",
                                     "lgbtq+ hip hop":"hip hop", "dfw rap": "hip hop", "contemporary country":"country"}})

In [ ]:
df_genre = songs_df['genre'].value_counts()[:20].sort_values(ascending=False) / len(songs_df)
sizes = df_genre.values.tolist()
labels = df_genre.index.values.tolist()
# Pie chart for genre
fig1, ax1 = plt.subplots(figsize=(10,10))
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=False, textprops={'fontsize': 14})
ax1.axis('equal')
plt.title("Most Popular Genres\n" + "(limited to top 10 genres)", bbox={'facecolor':'0.8', 'pad':5})
plt.show()

# dealing with outliers - later on!

In [ ]:
songs_df[songs_df['genre']=='ccm']

conclusion from the pie chart:
 pop, rap,hip hop and country are the most popular genres in the billaboard charts, by far from the other genres.

# dropping unnecessary features

In [ ]:
songs_df.columns

In [ ]:
#type has  only one value -  audio features , so we will drop it
songs_df['type']

In [ ]:
#'id', 'uri', 'track_href', 'analysis_url' aren't linked to the song genre at all, so we can drop them 

In [ ]:
songs_df=songs_df.drop(columns=['type','id', 'uri','track_href','analysis_url'])

# Dealing with Catagorials

In [ ]:
#save a copy before we changed the catagorials
songs_df_copy=songs_df.copy()

In [ ]:
songs_df.columns

the only Catagorial feature is out target variable - genre

since this is  Nominal , i.e there is no particular order between different values of genres and also this this is not a binary (yes/no) column, we will use  

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore',sparse=False)

In [ ]:
songs_df['genre'].unique()

In [ ]:
# pd.get_dummies(songs_df,columns=['genre'])

In [ ]:
# data_cols = pd.DataFrame(ohe.fit_transform(songs_df['genre']))

# data_cols

In [ ]:
#! pip install category_encoders

In [ ]:
#A set of scikit-learn-style transformers for encoding categorical variables into numeric with different techniques.
import category_encoders as ce

In [ ]:
# # def target_encode_multiclass(X1,y1): #X1,y1 are pandas df and series    
# X1=songs_df.drop(['genre'],axis=1)
# y1=songs_df['genre'] 

# enc=ce.OneHotEncoder().fit(songs_df['genre'].astype(str))
# y_onehot=enc.transform(songs_df['genre'].astype(str))
# # print(y_onehot)
# enc.get_feature_names()
# # pd.concat([songs_df,y_onehot])
# songs_df=songs_df.join(y_onehot)



In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder(sparse=False)
# ohe.fit_transform(songs_df[['genre']])

songs_df['genre']

In [ ]:
# onehot_encoder= OneHotEncoder(categories=songs_df['genre']) 

In [ ]:
# dummies_summary = pd.get_dummies(weather['Summary'])
# merged_summary = pd.concat([weather,dummies_summary],axis='columns')
# weather = merged_summary.drop(['Summary'], axis='columns')
# weather

In [ ]:
from sklearn.preprocessing import OneHotEncoder

dummies_genre=pd.get_dummies(songs_df['genre'])
merged_genre = pd.concat([songs_df,dummies_genre],axis='columns')
songs_df = merged_genre
songs_df

In [ ]:
songs_df

In [ ]:
songs_df['genre']

In [ ]:
# X1=songs_df.drop(['genre'],axis=1)
# y1=songs_df['genre']                 
# target_encode_multiclass(X1,y1)

# train test split

In [ ]:
X = songs_df.drop(['genre'],axis=1)
Y = songs_df['genre']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20,random_state=42)

In [ ]:
X

# Visualisation

In [ ]:
Y

In [ ]:
X_train.columns

In [ ]:
numerical_features=['danceability','energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'time_signature', 'popularity', 'rank']

In [ ]:
train_numeric=X_train[numerical_features]

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(X_train[columns=[numerical_features]].corr(),annot=True);

features that have high correlation:
    
    *loudness and energy  - the more a song is loud, it has more energy 
    
    * pop and rank - the more the song is popular, the more its' rank is high
    
     * danceability  and speechiness - 
    
    * acousticness and mode - 
    
   * acousticness and danceability 
    
   *acousticness and energy 
   
   * loudness and valance - the more a song is loud, its' valance is high ( it is more happy and cheerful)
   
   some of these correlations are very intuitive

In [ ]:
# corr_matrix = X_train.corr()
# corralations = corr_matrix['genre'].sort_values(ascending = False) 
# high_corr = (corralations > 0.2)|(corralations < -0.2)
# pd.DataFrame(corralations[high_corr])
# corralations[high_corr].index

#   מה 25.2 טיוטות

In [ ]:
songs={'Alice in Chains':'Them Bones','Beyonce':'Crazy In Love (feat. Jay-Z)','Glass Animals':'Heat Waves','Ed Sheeran':'The Joker And The Queen (feat. Taylor Swift)'}
song_names=songs.values()
artists_names=songs.keys()
song_names_list=list(song_names)

In [ ]:
#finding the id of the specific song with the song name and a dictionary that contains the artists info
def find_id(song,result):
    for data in result['tracks']['items']:
        for key,value in data.items():
            if (key=='name')&  (value==song):
                print(data['id'])
                print(data['uri'])
                print(sp.audio_features(data['uri'])[0])
                print(sp.artist(find_artist_info(result)))
                return 
               

In [ ]:
# unneccesary
# def get_key(val):
#     for key, value in songs.items():
#          if val == value:
#              return key
 
#     return "key doesn't exist"

In [ ]:
# unnessecay
# def get_id_of_songs():

#     # for song in song_names_list:

#     for song in song_names_list:
#         result = sp.search(get_key(song), limit=10)
#      #   print(result)
#         print("\n")
#         find_id(song,result)



In [ ]:
get_id_of_songs()

# טיוטות

In [ ]:
# for artist in artists_names:
#     art=sp.search(artist)

In [ ]:
#art

In [ ]:
#art['tracks']['items']

In [ ]:
#art['tracks']['items'][0]

In [ ]:
#art['tracks']['items']

In [ ]:
# #finding the id of the specific song with the song name and a dictionary that contains the artists info
# def find_id2(song,result):
#     for data in result['tracks']['items']:
#         for key,value in data.items():
#             if (key=='name')&  (value==song):
#                 print(data['id'])
#                 print(data['uri'])
#                 print(sp.audio_features(data['uri'])[0])
#                 print(sp.artist("64tNsm6TnZe2zpcMVMOoHL"))
#                 return 
               

In [ ]:

# find_id2('Them Bones',result)

In [ ]:
#  print(sp.artist(find_artist_id()))

In [ ]:

#  artists_names=songs.keys()
# def find_artist_id():
#         for key,value in art['tracks']['items'][0]['artists'][0].items():
#             if(key=='id'):
#                 print(value)
                   

In [ ]:
# def find_features(song,result):
#     for song in song_names_list:
#         result = sp.search(get_key(song), limit=10)
#         for data in result['tracks']['items']:
#             print(sp.audio_features("spotify:track:4A065x9kJt955eGVqf813g")[0])
#sp.artist("spotify:artist:6eUKZXaKkcviH0Ku9w2n3V")


In [ ]:
# get_id_of_songs()

In [ ]:
# art=sp.search(artist)

## כל מה שלמטה זה טיוטה

In [ ]:
# tracks = []

# playlist_link ="spotify:track:4A065x9kJt955eGVqf813g"
# #playlist_URI = playlist_link.split("/")[-1].split("?")[0]
# track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

# track_uri = track["track"]["uri"]

# #Track name
# track_name = track["track"]["name"]
    
# #Main Artist
# artist_uri = track["track"]["artists"][0]["uri"]
# artist_info = sp.artist(artist_uri)

# #Name, popularity, genre
# artist_name = track["track"]["artists"][0]["name"]
# artist_pop = artist_info["popularity"]
# artist_genres = artist_info["genres"]

# #Album
# album = track["track"]["album"]["name"]

# #Popularity of the track
# track_pop = track["track"]["popularity"]

# track_features = sp.audio_features(track_uri)[0]

# track_genre = 'None'
# if len(artist_genres)>0:
#     track_genre = artist_genres[0]

# track_features['genre'] = track_genre
# track_features['pop'] = track_pop
# track_features['name'] = track_name
# track_features['artist'] = artist_name
# tracks.append(track_features)

In [ ]:
# def find_artist_id0():
#     for song in song_names_list:
#         for data in art['tracks']['items']:
#             for key,value in data.items():
              
#                 if ((key=='uri') & (str(value).contains('a'))) :
#                     print(str(value))
#                     print (data['uri'])
#                     print('dgft')
#                     return
#                     #print(data['uri'])
        

In [ ]:
  #  print(result)
# for track in song_names_list:
#     #print(song_names_list[track])
#     if(track==result['tracks']['items'][0]['name']):
#         print(result['tracks']['items'][0]['name'])
# for item in a_dict.items():
# ...     print(item)

In [ ]:
# song_names_list

In [ ]:
# data.items()

In [ ]:
# result['tracks']

In [ ]:
# for data in result['tracks']['items']:
#     for key,value in data.items():
#         if (key=='name')&  (value==song_names_list[0]):
#             print(value)

In [ ]:
# result = sp.search(songs, type='track')

In [ ]:
# data.items()

In [ ]:
# song_names_list

In [ ]:
# result

In [ ]:
#result['tracks']['items'] - a list of dicts

In [ ]:
# result['tracks']['items']

In [ ]:
#print(result['tracks']['items'])

In [ ]:
# for data in result['tracks']['items']:
#     for key,value in data.items():
#         if (key=='name')&  (value==song_names_list[0]):
#             print(value)

In [ ]:
# for data in result['tracks']['items']:
#      for key in data:
#             if key=='name'& (value==song_names_list[0]):
#                 print(value)
   

In [ ]:
# for data in result['tracks']['items']:
#      for key in data.keys():
#             if key=='name':
#                 print(key)
   

In [ ]:
# for data in result['tracks']['items']:
#     print (data)